In [ ]:
# https://wikidocs.net/60691

# 02. IMDB 리뷰 감성 분류하기(IMDB Movie Review Sentiment Analysis)

참고 논문: http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf

In [1]:
# 영화 사이트 IMDB의 리뷰 데이터를 이용해 1 긍정 , 0 부정을 분류해보자

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
import random

In [3]:
# 랜덤시드 고정
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [13]:
# 하이퍼파라미터 변수
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

In [39]:
# 디바이스 설정
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

cpu와 cuda 중 다음 기기로 학습함: cpu
